# Generate response
response = rag_model.generate_content("Quelle est la première fable du livre ?")
print(response.text)

Convert PDF file to text file

Install required librairies

In [2]:
!pip install PyPDF2 google-cloud-storage

Import libraries

In [3]:
from google.cloud import storage
import PyPDF2
import os

Fonctions

In [4]:
# Fonction pour extraire le texte d'un PDF
def extract_text_from_pdf(pdf_path):
    """
    Extrait le texte d'un fichier PDF.
    """
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

In [5]:
def retrieve_pdf_files_from_source_bucket(bucket):

    # Lister les fichiers PDF dans le bucket
    pdf_files = [blob.name for blob in bucket.list_blobs() if blob.name.endswith(".pdf")]

    # Afficher la liste des fichiers PDF
    print("Fichiers PDF dans le bucket :")
    for pdf_file in pdf_files:
        print(pdf_file)
    
    return pdf_files

In [13]:
# Fonction pour télécharger un fichier depuis un bucket GCS
def download_from_gcs(source_bucket, source_blob_name, destination_file_name):
    """
    Télécharge un fichier depuis un bucket GCS.
    """
    blob = source_bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Fichier {source_blob_name} téléchargé depuis {source_bucket}.")

In [14]:
# Fonction pour téléverser un fichier dans un bucket GCS
def upload_to_gcs(destination_bucket, source_file_name, destination_blob_name):
    """
    Téléverse un fichier dans un bucket GCS.
    """
    blob = destination_bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"Fichier {source_file_name} téléversé dans {destination_bucket} sous le nom {destination_blob_name}.")

Pipeline principale

In [18]:
# Fonction principale pour la pipeline
def pdf_to_text_pipeline(source_bucket_name, destination_bucket_name, local_folder = "cours5"):
    """
    Pipeline pour convertir des fichiers PDF d'un bucket GCS en fichiers texte dans un autre bucket.
    
    Args:
        source_bucket_name: Nom du bucket source contenant les PDF.
        destination_bucket_name: Nom du bucket de destination pour les fichiers texte.
        local_folder: Dossier temporaire pour stocker les fichiers PDF téléchargés.
    """
    # Créer un dossier pour manipuler les fichiers pdf
    if not os.path.exists(local_folder):
        os.makedirs(local_folder)

    # Initialiser le client GCS et les buckets GCS
    storage_client = storage.Client()
    source_bucket = storage_client.bucket(source_bucket_name)
    destination_bucket = storage_client.bucket(destination_bucket_name)
    
    # Récupération des fichiers pdf du bucket source
    pdf_files = retrieve_pdf_files_from_source_bucket(source_bucket)
    
    # Traiter chaque fichier PDF
    for pdf_file in pdf_files:
        try:
            # Télécharger le PDF
            local_pdf_path = os.path.join(local_folder, pdf_file)
            download_from_gcs(source_bucket, pdf_file, local_pdf_path)
            
            # Extraire le texte du PDF
            text = extract_text_from_pdf(local_pdf_path)
            
            # Sauvegarder le texte dans un fichier temporaire
            txt_file = pdf_file.replace(".pdf", ".txt")
            local_txt_path = os.path.join(local_folder, txt_file)
            with open(local_txt_path, "w") as f:
                f.write(text)
            
            # Téléverser le fichier texte dans le bucket de destination
            upload_to_gcs(destination_bucket, local_txt_path, txt_file)
            
            # Supprimer les fichiers temporaires du local folder
            # os.remove(local_pdf_path)
            # os.remove(local_txt_path)
        
        except Exception as e:
            print(f"Erreur lors du traitement du fichier {pdf_file} : {e}")

    print("Pipeline terminée avec succès.")

Variables

In [16]:
source_bucket_name = "cours5bucket"
destination_bucket_name  = "cours5bucket"
local_folder = "cours5"

Run pipeline

In [19]:
pdf_to_text_pipeline(source_bucket_name, destination_bucket_name, local_folder = "cours5")

Fichiers PDF dans le bucket :
deepseek-r1 paper.pdf
Fichier deepseek-r1 paper.pdf téléchargé depuis <Bucket: cours5bucket>.
Fichier cours5/deepseek-r1 paper.txt téléversé dans <Bucket: cours5bucket> sous le nom deepseek-r1 paper.txt.
Pipeline terminée avec succès.
